In [1]:
import pandas as pd
import torch
import dgl
import pickle
from utils import HGT
import random

In [2]:
graph = dgl.load_graphs('training_data/graph.dgl')
graph = graph[0][0]

with open('training_data/train.obj', 'rb') as fp:
	train = pickle.load(fp)

with open('training_data/val.obj', 'rb') as fp:
	val = pickle.load(fp)

with open('training_data/test.obj', 'rb') as fp:
	test = pickle.load(fp)

In [3]:
graph.nodes('business').shape[0]

150243

In [4]:
edges = {}
for canonical_etype in graph.canonical_etypes:
    edges[canonical_etype] = graph.edges(etype=canonical_etype)

edges[('category', 'category_to_business', 'business')] = (graph.edges(etype='business_has_category')[1], graph.edges(etype='business_has_category')[0])
edges[('business', 'business_to_review', 'review')] = (graph.edges(etype='review_to_business')[1], graph.edges(etype='review_to_business')[0])
edges[('business', 'business_to_tip', 'tip')] = (graph.edges(etype='tip_to_business')[1], graph.edges(etype='tip_to_business')[0])
edges[('review', 'review_to_user', 'user')] = (graph.edges(etype='user_to_review')[1], graph.edges(etype='user_to_review')[0])
edges[('tip', 'tip_to_user', 'user')] = (graph.edges(etype='user_to_tip')[1], graph.edges(etype='user_to_tip')[0])

num_nodes_dict = {} 
for ntype in graph.ntypes:
    num_nodes_dict[ntype] = graph.nodes(ntype).shape[0]

g = dgl.heterograph(edges, num_nodes_dict = num_nodes_dict)
g.ndata['feat'] = {k: torch.tensor(v, dtype=torch.float32) for k, v in graph.ndata['feat'].items() }
del graph

/var/folders/r3/89932hws5rg2xwsqck66y26r0000gp/T/ipykernel_10585/4158148776.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  g.ndata['feat'] = {k: torch.tensor(v, dtype=torch.float32) for k, v in graph.ndata['feat'].items() }


In [5]:
g.num_nodes

<bound method DGLGraph.num_nodes of Graph(num_nodes={'business': 150243, 'category': 1311, 'review': 6339837, 'tip': 908878, 'user': 1987897},
      num_edges={('business', 'business_has_category', 'category'): 668592, ('business', 'business_to_review', 'review'): 6339837, ('business', 'business_to_tip', 'tip'): 908878, ('category', 'category_to_business', 'business'): 668592, ('review', 'review_to_business', 'business'): 6339837, ('review', 'review_to_user', 'user'): 6339837, ('tip', 'tip_to_business', 'business'): 908878, ('tip', 'tip_to_user', 'user'): 908878, ('user', 'user_to_review', 'review'): 6339837, ('user', 'user_to_tip', 'tip'): 908878, ('user', 'user_to_user', 'user'): 437928},
      metagraph=[('business', 'category', 'business_has_category'), ('business', 'review', 'business_to_review'), ('business', 'tip', 'business_to_tip'), ('category', 'business', 'category_to_business'), ('review', 'business', 'review_to_business'), ('review', 'user', 'review_to_user'), ('tip', 'bus

In [6]:
node_dict = { ntype: g.ntypes.index(ntype) for ntype in g.ntypes }
edge_dict = { canonical_etype: g.canonical_etypes.index(canonical_etype) for canonical_etype in g.canonical_etypes }
feature_dim_dict = { ntype: g.ndata['feat'][ntype].shape[1] for ntype in g.ntypes }

In [7]:
node_dict

{'business': 0, 'category': 1, 'review': 2, 'tip': 3, 'user': 4}

In [8]:
model = HGT(node_dict, edge_dict, feature_dim_dict, n_hid=256, n_out=128, n_layers=4, n_heads=8, use_norm=True)
opt = torch.optim.AdamW(model.parameters(), 1e-4)
sampler = dgl.dataloading.NeighborSampler([100, 100, 100, 100])
# dgl.dataloading.NeighborSampler([
#     {('user', 'follows', 'user'): 5,
#      ('user', 'plays', 'game'): 4,
#      ('game', 'played-by', 'user'): 3}] * 3)

business
category
review
tip
user


In [9]:
train_pos_ids = list(range(train['pos'][0].shape[0]))
train_neg_ids = list(range(train['neg'][0].shape[0]))

In [10]:
len(train_neg_ids)

135108

In [11]:
g.ndata['feat']['category'].dtype

torch.float32

In [12]:
def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]
batch_size = 64
for epoch in range(1):
    model.train()
    random.shuffle(train_pos_ids)
    random.shuffle(train_neg_ids)
    for batch in split(list(zip(train_pos_ids, train_neg_ids)), batch_size):
        opt.zero_grad()
        pos_ids, neg_ids = list(zip(*batch))
        pos_block_user = [blocks for _, _, blocks in dgl.dataloading.DataLoader(
            g, {'user': torch.index_select(train['pos'][0], 0, torch.tensor(pos_ids))}, sampler,
            batch_size=batch_size, shuffle=False, drop_last=False, num_workers=4)][0]
        
        pos_block_business = [blocks for _, _, blocks in dgl.dataloading.DataLoader(
            g, {'business': torch.index_select(train['pos'][1], 0, torch.tensor(pos_ids))}, sampler,
            batch_size=batch_size, shuffle=False, drop_last=False, num_workers=4)][0]
        
        neg_block_user = [blocks for _, _, blocks in dgl.dataloading.DataLoader(
            g,{'user': torch.index_select(train['neg'][0], 0, torch.tensor(neg_ids))}, sampler,
            batch_size=batch_size, shuffle=False, drop_last=False, num_workers=4)][0]
        
        neg_block_business = [blocks for _, _, blocks in dgl.dataloading.DataLoader(
            g, {'business': torch.index_select(train['neg'][1], 0, torch.tensor(neg_ids))}, sampler,
            batch_size=batch_size, shuffle=False, drop_last=False, num_workers=4)][0]
        pos_user_logits =  model(pos_block_user, 'user')
        pos_business_logits =  model(pos_block_business, 'business')
        neg_user_logits =  model(neg_block_user, 'user')
        neg_business_logits =  model(neg_block_business, 'business')
        break
    break

/Users/filippominutella/miniconda3/lib/python3.10/site-packages/dgl/dataloading/dataloader.py:869: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(f'Dataloader CPU affinity opt is not enabled, consider switching it on '


[Block(num_src_nodes={'business': 33949, 'category': 499, 'review': 128416, 'tip': 40386, 'user': 60501},
      num_dst_nodes={'business': 1270, 'category': 393, 'review': 63756, 'tip': 23681, 'user': 394},
      num_edges={('business', 'business_has_category', 'category'): 36612, ('business', 'business_to_review', 'review'): 63756, ('business', 'business_to_tip', 'tip'): 23681, ('category', 'category_to_business', 'business'): 6542, ('review', 'review_to_business', 'business'): 96168, ('review', 'review_to_user', 'user'): 6307, ('tip', 'tip_to_business', 'business'): 37142, ('tip', 'tip_to_user', 'user'): 509, ('user', 'user_to_review', 'review'): 63756, ('user', 'user_to_tip', 'tip'): 23681, ('user', 'user_to_user', 'user'): 593},
      metagraph=[('business', 'category', 'business_has_category'), ('business', 'review', 'business_to_review'), ('business', 'tip', 'business_to_tip'), ('category', 'business', 'category_to_business'), ('review', 'business', 'review_to_business'), ('revie

/Users/filippominutella/miniconda3/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


{'src': {'business': tensor([], size=(0, 256), grad_fn=<NativeLayerNormBackward0>), 'category': tensor([], size=(0, 256), grad_fn=<NativeLayerNormBackward0>), 'review': tensor([], size=(0, 256), grad_fn=<NativeLayerNormBackward0>), 'tip': tensor([], size=(0, 256), grad_fn=<NativeLayerNormBackward0>), 'user': tensor([[-0.5383,  0.1344,  1.9124,  ..., -0.8867,  2.5933, -0.3757],
        [-0.3476,  0.4913,  1.5305,  ..., -0.8310,  3.4158, -0.6781],
        [-0.8630,  0.2848,  0.6960,  ..., -0.9120, -0.5032,  0.2860],
        ...,
        [-0.8188,  0.5182,  0.8998,  ..., -0.8186,  0.7291, -0.0154],
        [ 1.4346, -0.4769,  2.1302,  ..., -0.9533,  3.0960, -0.8752],
        [-0.8189,  0.8038,  1.2316,  ..., -0.8768,  2.4058, -0.6116]],
       grad_fn=<NativeLayerNormBackward0>)}, 'dst': {}}
{'src': {'business': tensor([[ 2.2826, -0.6616,  0.3384,  ..., -0.5540,  2.6382, -0.7411],
        [ 1.6228, -0.7555,  2.0287,  ..., -0.7953,  1.4241, -0.8384],
        [ 2.2896, -0.6736,  0.9918,  ..

: 

: 